In [1]:
import pandas as pd
from utils_analysis import prepare_scores
import matplotlib.pyplot as plt
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [45]:
df = pd.read_csv("data/debatesum/test.csv")

# "results/bart/r1/bart-base_data_debatesum_checkpoint-1500_debatesum.json",
# "results/bart/r2/bart-base_data_debatesum_checkpoint-1500_debatesum.json",
# "results/bart/r3/bart-base_data_debatesum_checkpoint-1500_debatesum.json",
# "results/t5/r1/flan-t5-base_data_debatesum_checkpoint-1500_debatesum.json",
# "results/t5/r2/flan-t5-base_data_debatesum_checkpoint-1500_debatesum.json",
# "results/t5/r3/flan-t5-base_data_debatesum_checkpoint-1500_debatesum.json",

df_scores, beam_search_results = prepare_scores(
    "results/t5/r1/flan-t5-base_data_debatesum_checkpoint-1500_debatesum.json",
    df["target"],
    "rougeL",
)

/home/mila/f/floresl/beam-search/utils_analysis.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  abs(spearmanr(ground_truth_score, confidence_score_dict[str(k)]).statistic)  # type: ignore


In [46]:
beam_score_log_probs = pd.DataFrame(beam_search_results["beam_score_log_probs"])
worst_rouge_idxs = df_scores.sort_values(by="rougeL").index[:100]
best_rouge_idxs = df_scores.sort_values(by="rougeL").index[::-1][:100]

In [47]:
# plt.bar(
#     x=list(range(1, 101)),
#     height=np.exp(beam_score_log_probs.loc[worst_rouge_idxs[0]]),
# )
# plt.xlabel("Beam Number")
# plt.ylabel("Joint Sequence Token Probability")

In [48]:
# plt.bar(
#     x=list(range(1, 101)),
#     height=np.exp(beam_score_log_probs.loc[best_rouge_idxs[-1]]),
# )
# plt.xlabel("Beam Number")
# plt.ylabel("Joint Sequence Token Probability")

In [49]:
df_scores.drop(["sentences", "dropout_sentences"], axis=1).corr(method="spearman")[
    "rougeL"
]

length_normalized_log_probs   -0.247232
mean_token_entropy             0.247735
dropout_bleu_variance         -0.009334
dropout_meteor_score           0.042772
dropout_entropy                0.031585
dropout_disagreement           0.083677
rougeL                         1.000000
beam_score_ratios_94           0.295849
beam_score_log_probs_94       -0.257584
beam_score_top_k_94           -0.253605
beam_score_impt_wt_94          0.253581
Name: rougeL, dtype: float64

In [50]:
from utils_tail_probs import softmax, tail_index
from scipy.stats import spearmanr

In [51]:
tail_indices = []
for i in range(len(beam_score_log_probs)):
    probs = softmax(beam_score_log_probs.iloc[i].to_numpy(), temperature=1)
    tail_indices.append(tail_index(probs))

spearmanr(tail_indices, df_scores["rougeL"])

SignificanceResult(statistic=0.3540283266320829, pvalue=6.745576660930582e-31)

In [2]:
import json

baseline_probs = json.load(
    open(
        "/home/mila/f/floresl/beam-search/results/bart_baseline/facebook_bart-base_debatesum.json",
        "r",
    )
)

In [53]:
baseline_log_probs = baseline_probs["beam_score_log_probs"]
baseline_log_probs = pd.DataFrame(baseline_log_probs).values

beam_score_log_probs = beam_score_log_probs.values

In [54]:
from scipy.spatial.distance import jensenshannon

In [55]:
js_distances = []
for i in range(1000):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=1),
        softmax(baseline_log_probs[i], temperature=1),
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["rougeL"])

SignificanceResult(statistic=0.3516849511863946, pvalue=1.7426765223803198e-30)

In [56]:
js_distances = []
for i in range(1000):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=1), np.array([1 / 100] * 100)
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["rougeL"])

SignificanceResult(statistic=0.35424229417383385, pvalue=6.183140771729131e-31)